In [1]:
!pip install -q "sagemaker>=2.70.0" "transformers==4.11.0" --upgrade

!pip install -q "datasets==1.13" --upgrade


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.22 requires botocore==1.29.22, but you have botocore 1.29.23 which is incompatible.
aiobotocore 2.0.1 requires botocore<1.22.9,>=1.22.8, but you have botocore 1.29.23 which is incompatible.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install -q torch

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [3]:
import numpy as np
import random
import pandas as pd

In [4]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

# Data Preprocessing

In [5]:
dataset = fetch_20newsgroups(subset="all", shuffle=True, remove=("headers", "footers", "quotes"))

In [6]:
documents = dataset.data
labels = dataset.target
df=pd.DataFrame({"documents":documents,"labels":labels})

In [7]:
df

,documents,labels
0,\n\nI am sure some bashers of Pens fans are pr...,10
1,My brother is in the market for a high-perform...,3
2,\n\n\n\n\tFinally you said what you dream abou...,17
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3
4,1) I have an old Jasmine drive which I cann...,4
...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,13
18842,\nNot in isolated ground recepticles (usually ...,12
18843,I just installed a DX2-66 CPU in a clone mothe...,3
18844,\nWouldn't this require a hyper-sphere. In 3-...,1


In [8]:
import re,string

def text_cleaner(text):

    '''some text cleaning method'''

    text = text.lower()

    text = re.sub('\[.*?\]', '', text)

    text = re.sub('https?://\S+|www\.\S+', '', text)

    text = re.sub('<.*?>+', '', text)

    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)

    text = re.sub('\n', '', text)

    text = re.sub('\w*\d\w*', '', text)

    return text

In [9]:
df["documents"]=df["documents"].apply(text_cleaner)

In [10]:
df=df.head(1100)

In [11]:
df.shape

(1100, 2)

In [12]:
df

,documents,labels
0,i am sure some bashers of pens fans are pretty...,10
1,my brother is in the market for a highperforma...,3
2,\tfinally you said what you dream about medite...,17
3,thinkits the scsi card doing the dma transfers...,3
4,i have an old jasmine drive which i cannot...,4
...,...,...
1095,i am hoping to produce the first update of the...,9
1096,in article manesmagpielinknetcom steve i...,16
1097,sorry malcolm but i rather believe jesus than...,19
1098,i have placed a new release of my axe editor i...,5


In [13]:
df = df.sample(frac = 1)

In [14]:
df

,documents,labels
456,anyone familiar with this video card what chi...,3
1039,yeah it might if you only read the part you qu...,13
1036,im no defender of the aec but it is worth noti...,18
597,,8
75,phil didnt one of the early jet fighters have ...,14
...,...,...
447,the problem is that the pins in the adb connec...,4
36,would you believe that there is a letter i...,4
479,\tim not sure that you can distinguish between...,19
591,some of the davidians are blacknext question,16


In [15]:
rows = df.shape[0]
train = int(.8* rows)
test = rows-train

In [16]:
df[:train].shape,df[train:].shape

((880, 2), (220, 2))

In [17]:
df[:train].to_csv('train.csv'
                          ,index=False)

In [18]:
# Write Validation Set
df[train:].to_csv('test.csv'
                          ,index=False)

In [19]:
df

,documents,labels
456,anyone familiar with this video card what chi...,3
1039,yeah it might if you only read the part you qu...,13
1036,im no defender of the aec but it is worth noti...,18
597,,8
75,phil didnt one of the early jet fighters have ...,14
...,...,...
447,the problem is that the pins in the adb connec...,4
36,would you believe that there is a letter i...,4
479,\tim not sure that you can distinguish between...,19
591,some of the davidians are blacknext question,16


In [20]:
df[["labels"]]

,labels
456,3
1039,13
1036,18
597,8
75,14
...,...
447,4
36,4
479,19
591,16


# Sagemaker

In [21]:
import sagemaker
from sagemaker import get_execution_role
# from sagemaker.sklearn.estimator import SKLearn
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_session.region_name
import os
import boto3

In [22]:
region

'us-east-2'

In [23]:
bucket_name = 'agtimeseries'

training_folder = r'mce-folder/train'
test_folder = r'mceolder/test'
model_folder1 = r'mce-hf-folder/model1/'
script_code_folder1=r"mce-folder/scriptcode1"

model_folder2 = r'mce-folder/model2/'
script_code_folder2=r"mce-folder/scriptcode2"

training_data_loc = r's3://' + bucket_name + r'/' + training_folder
testing_data_loc = r's3://' + bucket_name + r'/' + test_folder
model_data_loc1 = r's3://' + bucket_name + r'/' + model_folder1
script_code_loc1=r's3://' + bucket_name + r'/' + script_code_folder1


model_data_loc2 = r's3://' + bucket_name + r'/' + model_folder2
script_code_loc2=r's3://' + bucket_name + r'/' + script_code_folder2

In [24]:
training_data_loc

's3://agtimeseries/mce-folder/train'

In [25]:
train_file="train.csv"
test_file="test.csv"

In [26]:
sagemaker_session.upload_data(train_file,
                              bucket=bucket_name, 
                              key_prefix=training_folder)

's3://agtimeseries/mce-folder/train/train.csv'

In [27]:
sagemaker_session.upload_data(test_file, 
                              bucket=bucket_name, 
                              key_prefix=test_folder)

's3://agtimeseries/mceolder/test/test.csv'

In [28]:
model_data_loc1

's3://agtimeseries/mce-hf-folder/model1/'

In [29]:
from sagemaker.huggingface import HuggingFace,TrainingCompilerConfig

In [30]:
compiler_config=TrainingCompilerConfig()

# HuggingFace Estimator:

In [31]:
hyperparameters={'epochs': 1,                                    # number of training epochs
                 'train_batch_size': 8,                         # batch size for training
                 'eval_batch_size': 8,                          #batch size for testing
                 "model_id" :'bert-base-uncased'                #pretrained model name from hf
                }

In [32]:
huggingface_estimator1 = HuggingFace(
    entry_point          = 'script.py',       # script code to run the model      
    source_dir           = 'Hfcode/',
    instance_type        = 'ml.g4dn.xlarge',    # instances type used for the training job
    instance_count       = 1,                  # the number of instances used for training
    base_job_name        = "bert-model",       # the name of the training job     
    role                 = role,               # Iam role used in training job to access AWS ressources, e.g. S3
    transformers_version = '4.11.0',           # the transformers version used in the training job
    pytorch_version      = '1.9.0',            # the pytorch_version version used in the training job
    py_version           = 'py38',             # the python version used in the training job
    compiler_config      = compiler_config,    # the compiler configuration used in the training job
    hyperparameters      = hyperparameters,    # the hyperparameter used for running the training job
    disable_profiler     = True,               # whether to disable the profiler during training used to gain maximum performance
    debugger_hook_config = False,              # whether to enable the debugger hook during training used to gain maximum performance
    output_path          = model_data_loc1,     # s3 location where the final fine-tuned model will be saved 
    code_location        = script_code_loc1     # s3 location where the estimator will push the script code
)

In [33]:
data={'training': training_data_loc, 'testing': testing_data_loc}

In [34]:
huggingface_estimator1.fit(data)

2022-12-06 05:42:55 Starting - Starting the training job...
2022-12-06 05:43:10 Starting - Preparing the instances for training......
2022-12-06 05:44:11 Downloading - Downloading input data...
2022-12-06 05:44:31 Training - Downloading the training image..............................
2022-12-06 05:49:38 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-12-06 05:49:50,783 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-12-06 05:49:50,808 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-12-06 05:49:50,812 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-12-06 05:49:51,079 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {
        "sagemaker_training_compiler_debug_mode

100%|██████████| 107/107 [04:05<00:00,  4.98s/it]
***** Running Evaluation *****
Num examples = 213
  Batch size = 8
***** Running Evaluation *****
  Num examples = 213
  Batch size = 8
100%|██████████| 27/27 [00:09<00:00,  1.15s/it]#033[A
#033[A
{'eval_loss': 2.48561429977417, 'eval_accuracy': 0.2863849765258216, 'eval_runtime': 13.0248, 'eval_samples_per_second': 16.353, 'eval_steps_per_second': 2.073, 'epoch': 1.0}
#015100%|██████████| 27/27 [00:09<00:00,  1.15s/it]#033[A
#015                                               #033[A
Saving model checkpoint to ./results/checkpoint-107
Saving model checkpoint to ./results/checkpoint-107
Configuration saved in ./results/checkpoint-107/config.json
Configuration saved in ./results/checkpoint-107/config.json


Model weights saved in ./results/checkpoint-107/pytorch_model.bin
Model weights saved in ./results/checkpoint-107/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-107/tokenizer_config.json
tokenizer config file saved in ./results/checkpoint-107/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-107/special_tokens_map.json
Special tokens file saved in ./results/checkpoint-107/special_tokens_map.json
Training completed. Do not forget to share your model on huggingface.co/models =)
Loading best model from ./results/checkpoint-107 (score: 2.48561429977417).
Training completed. Do not forget to share your model on huggingface.co/models =)
Loading best model from ./results/checkpoint-107 (score: 2.48561429977417).
{'train_runtime': 263.8688, 'train_samples_per_second': 3.244, 'train_steps_per_second': 0.406, 'train_loss': 2.7922003915376754, 'epoch': 1.0}
100%|██████████| 107/107 [04:23<00:00,  2.47s/it]
***** Running Evaluation *****
  Num examples 

In [35]:
huggingface_estimator1.model_data

's3://agtimeseries/mce-hf-folder/model1/bert-model-2022-12-06-05-42-55-321/output/model.tar.gz'

# Pytorch Estimator:

In [36]:
from sagemaker.pytorch import PyTorch as PyTorchEstimator

In [37]:
hyperparameters={'epochs': 1,                                    # number of training epochs
                 'train_batch_size': 8,                         # batch size for training
                 'eval_batch_size': 8,                          #batch size for testing
                 "model_id" :'bert-base-uncased'                #pretrained model name from hf
                }

In [38]:
pytorch_estimator1 = PyTorchEstimator(
    entry_point          = 'script.py',       # script code to run the model
    source_dir           = 'Pcode/',
    instance_type        = 'ml.g4dn.xlarge',    # instances type used for the training job
    instance_count       = 1,                  # the number of instances used for training
    base_job_name        = "bert-model",       # the name of the training job     
    role                 = role,               # Iam role used in training job to access AWS ressources, e.g. S3
    py_version='py38',                           # the python version used in the training job
    framework_version='1.9.0'   ,              # the pytorch_version version used in the training job       
    hyperparameters      = hyperparameters,    # the hyperparameter used for running the training job
    disable_profiler     = True,               # whether to disable the profiler during training used to gain maximum performance
    debugger_hook_config = False,              # whether to enable the debugger hook during training used to gain maximum performance
    output_path          = model_data_loc2,     # s3 location where the final fine-tuned model will be saved 
    code_location        = script_code_loc2     # s3 location where the estimator will push the script code
)

In [39]:
data={'training': training_data_loc, 'testing': testing_data_loc}

In [40]:
pytorch_estimator1.fit(data)

2022-12-06 06:28:18 Starting - Starting the training job...
2022-12-06 06:28:33 Starting - Preparing the instances for training......
2022-12-06 06:29:48 Downloading - Downloading input data
2022-12-06 06:29:48 Training - Downloading the training image..............................
2022-12-06 06:34:39 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-12-06 06:34:52,002 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-12-06 06:34:52,028 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-12-06 06:34:52,031 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-12-06 06:34:52,255 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "testing": "

Preparing metadata (setup.py): started
Preparing metadata (setup.py): finished with status 'done'
Preparing metadata (setup.py): started
Preparing metadata (setup.py): finished with status 'done'
Preparing metadata (setup.py): started
Preparing metadata (setup.py): finished with status 'done'
Preparing metadata (setup.py): started
Preparing metadata (setup.py): finished with status 'done'
Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=3732e91f24fec1f2b7efc44c676789a1bbd4c984931923103625679ab401f503
Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732


Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21502 sha256=4dc6131383ceb00ed56f432dbd3480fd690943a75a89ac169abcace3222a1f85
Stored in directory: /root/.cache/pip/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=401a915acf6964bcafec46b5d5988f15c6b280c373447e9ea7e1351c40aaa404
Stored in directory: /root/.cache/pip/wheels/19/09/72/3eb74d236bb48bd0f3c6c3c83e4e0c5bbfcbcad7c6c3539db8
Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=837b7bb404dab5865307101f5dcdb7311145916dc2766e9ef1e2a7fe965552cc
Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built seqeval promise validators pathtools
Attempting uninstall: urllib3
Found existing installation: urllib3 1.26.6
Uninstalling urllib3-1.26.6:
Successfully uninstalled urllib3-1.26.6
Attempting uninstall: typing

Epochs 0/1. Running Loss:    2.6978: 100%|██████████| 107/107 [00:13<00:00,  7.80it/s]


/opt/conda/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
0%|          | 0/213 [00:00<?, ?it/s]
#033[A
0%|          | 1/213 [00:00<00:25,  8.40it/s]
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_20_2
Epoch 1 of 1: 100%|██████████| 1/1 [00:22<00:00, 22.26s/it]
INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.
2022-12-06 06:36:03,089 sagemaker-training-toolkit INFO     Reporting training SUCCESS

2022-12-06 06:36:20 Uploading - Uploading generated training model
2022-12-06 06:37:21 Completed - Training job completed
Training seconds: 472
Billable seconds: 472


In [41]:
pt_mnist_model_data=pytorch_estimator1.model_data

In [42]:
from sagemaker.s3 import S3Downloader

In [39]:
S3Downloader.download(pt_mnist_model_data, ".")

# Endpoint Name

In [43]:
import boto3
from time import gmtime, strftime
CUR = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
ENDPOINT_NAME = "mme-hf-POC-V1" + "-" + CUR

MODEL_NAME = ENDPOINT_NAME

In [44]:
ENDPOINT_NAME

'mme-hf-POC-V1-2022-12-06-06-39-30'

# Create Model, Endpoint Config, Endpoint

In [45]:
pt_model_uri=pytorch_estimator1.model_data

In [46]:
pt_model_uri

's3://agtimeseries/mce-folder/model2/bert-model-2022-12-06-06-28-18-002/output/model.tar.gz'

In [47]:
pt_ecr_image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=region,
    version="1.9.0",
    py_version="py38",
    instance_type="ml.c5.4xlarge",
    image_scope="inference",
    container_version="ubuntu20.04"
)

In [48]:
pt_ecr_image_uri

'763104351884.dkr.ecr.us-east-2.amazonaws.com/pytorch-inference:1.9.0-cpu-py38-ubuntu20.04'

In [49]:
pytorch_container = {
    "ContainerHostname": "pytorch-model",
    "Image": pt_ecr_image_uri,
    "ModelDataUrl": pt_model_uri,
    "Environment": {
        "SAGEMAKER_PROGRAM": "inference.py",
        "SAGEMAKER_SUBMIT_DIRECTORY": pt_model_uri,
    },
}

In [50]:
pytorch_container

{'ContainerHostname': 'pytorch-model',
 'Image': '763104351884.dkr.ecr.us-east-2.amazonaws.com/pytorch-inference:1.9.0-cpu-py38-ubuntu20.04',
 'ModelDataUrl': 's3://agtimeseries/mce-folder/model2/bert-model-2022-12-06-06-28-18-002/output/model.tar.gz',
 'Environment': {'SAGEMAKER_PROGRAM': 'inference.py',
  'SAGEMAKER_SUBMIT_DIRECTORY': 's3://agtimeseries/mce-folder/model2/bert-model-2022-12-06-06-28-18-002/output/model.tar.gz'}}

In [51]:
hf_model_uri=huggingface_estimator1.model_data

In [56]:
sm_client = boto3.client("sagemaker")
# image_uri will depend on the region
image_uri = "763104351884.dkr.ecr.us-east-2.amazonaws.com/huggingface-pytorch-inference:1.9.0-transformers4.11.0-cpu-py38-ubuntu20.04"
primary_container_hf = {
    "ContainerHostname": "huggingface-model",
    'Image': image_uri,
    'ModelDataUrl': hf_model_uri,
    'Environment': {
        'SAGEMAKER_PROGRAM': 'inference.py',
        'SAGEMAKER_REGION': region,
        'SAGEMAKER_SUBMIT_DIRECTORY': hf_model_uri
    }
}


In [57]:
primary_container_hf

{'ContainerHostname': 'huggingface-model',
 'Image': '763104351884.dkr.ecr.us-east-2.amazonaws.com/huggingface-pytorch-inference:1.9.0-transformers4.11.0-cpu-py38-ubuntu20.04',
 'ModelDataUrl': 's3://agtimeseries/mce-hf-folder/model1/bert-model-2022-12-06-05-42-55-321/output/model.tar.gz',
 'Environment': {'SAGEMAKER_PROGRAM': 'inference.py',
  'SAGEMAKER_REGION': 'us-east-2',
  'SAGEMAKER_SUBMIT_DIRECTORY': 's3://agtimeseries/mce-hf-folder/model1/bert-model-2022-12-06-05-42-55-321/output/model.tar.gz'}}

In [58]:
ENDPOINT_NAME

'mme-hf-POC-V1-2022-12-06-06-39-30'

In [59]:
create_model_response = sm_client.create_model(
    ModelName=ENDPOINT_NAME,
    #multi-container-example
    Containers=[pytorch_container, primary_container_hf],
    InferenceExecutionConfig={"Mode": "Direct"},
    ExecutionRoleArn=role,
)

In [60]:
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName =ENDPOINT_NAME,
    ProductionVariants=[
        {
        'InstanceType':'ml.m5.4xlarge',
        'InitialInstanceCount':1,
        'ModelName': ENDPOINT_NAME,
        'VariantName':'AllTraffic',
        'InitialVariantWeight':1
        }
    ])
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

Endpoint configuration arn:  arn:aws:sagemaker:us-east-2:831536787935:endpoint-config/mme-hf-poc-v1-2022-12-06-06-39-30


In [61]:
endpoint_params = {
    'EndpointName': ENDPOINT_NAME,
    'EndpointConfigName':ENDPOINT_NAME,
}
endpoint_response = sm_client.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

EndpointArn = arn:aws:sagemaker:us-east-2:831536787935:endpoint/mme-hf-poc-v1-2022-12-06-06-39-30


In [62]:
waiter=sm_client.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=ENDPOINT_NAME)

# Endpoint Test

In [63]:
sen1='the universe mirrored in a puddleisnt it amazing how there always seems to be another bottle of bheer therealeph one bottles of beer on the wall aleph one null bottles of beer\tyou too are a puddle\tas above so below'
sen2='the universe mirrored in a puddleisnt it amazing how there always seems to be another bottle of bheer therealeph one bottles of beer on the wall aleph one null bottles of beer\tyou too are a puddle\tas above so below'

In [64]:
v=[{"text":sen1,"id":1},{"text":sen2,"id":2}]

In [65]:
import boto3
import json
client = boto3.client('sagemaker-runtime')

In [66]:
import boto3
import json
client = boto3.client('sagemaker-runtime')

response = client.invoke_endpoint(
EndpointName=ENDPOINT_NAME,
ContentType="application/json",
Accept="application/json",
TargetContainerHostname="huggingface-model",
Body=json.dumps(v)
)
print(response['Body'].read().decode())

[
  {
    "score": 0.07771327346563339,
    "Categ": 14,
    "pred": "i1",
    "probabs": {
      "0": 0.062302861362695694,
      "1": 0.02041207253932953,
      "2": 0.06461289525032043,
      "3": 0.03423217311501503,
      "4": 0.044934581965208054,
      "5": 0.024256065487861633,
      "6": 0.036553822457790375,
      "7": 0.06341090053319931,
      "8": 0.07290441542863846,
      "9": 0.031524669378995895,
      "10": 0.03600924089550972,
      "11": 0.04315823316574097,
      "12": 0.038719259202480316,
      "13": 0.05332612246274948,
      "14": 0.07771327346563339,
      "15": 0.07034662365913391,
      "16": 0.029433589428663254,
      "17": 0.0674159973859787,
      "18": 0.06579737365245819,
      "19": 0.06293588876724243
    },
    "text": "the universe mirrored in a puddleisnt it amazing how there always seems to be another bottle of bheer therealeph one bottles of beer on the wall aleph one null bottles of beer\tyou too are a puddle\tas above so below",
    "id": 1
  

In [67]:
import boto3
import json
client = boto3.client('sagemaker-runtime')

response = client.invoke_endpoint(
EndpointName=ENDPOINT_NAME,
ContentType="application/json",
Accept="application/json",
TargetContainerHostname="pytorch-model",
Body=json.dumps(v)
)
print(response['Body'].read().decode())

[
  {
    "score": "0.09843205895225313",
    "categ": "13",
    "pred": "n",
    "probabs": {
      "0": "0.03503072614401855",
      "1": "0.023843549854156376",
      "2": "0.023444839068190566",
      "3": "0.03545735012186487",
      "4": "0.038312047150484814",
      "5": "0.02553160807235324",
      "6": "0.035870460194183586",
      "7": "0.06402122147628944",
      "8": "0.0908671579613727",
      "9": "0.04136967595813908",
      "10": "0.0731020017872005",
      "11": "0.0515978588698538",
      "12": "0.04358914971715622",
      "13": "0.09843205895225313",
      "14": "0.0694045428263539",
      "15": "0.046193877917905725",
      "16": "0.0269933735583257",
      "17": "0.04977644705082381",
      "18": "0.06078476619048829",
      "19": "0.06637728712858572"
    },
    "text": "the universe mirrored in a puddleisnt it amazing how there always seems to be another bottle of bheer therealeph one bottles of beer on the wall aleph one null bottles of beer\tyou too are a puddl

In [54]:
import boto3
import json
client = boto3.client('sagemaker-runtime')

response = client.invoke_endpoint(
EndpointName=ENDPOINT_NAME,
ContentType="application/json",
Accept="application/json",
TargetModel="model1.tar.gz",
Body=json.dumps(v)
)
print(response['Body'].read().decode())

[
  {
    "score": 0.08863607794046402,
    "Categ": 19,
    "pred": "n1",
    "probabs": {
      "0": 0.0542377345263958,
      "1": 0.04734285920858383,
      "2": 0.02886742539703846,
      "3": 0.03112703375518322,
      "4": 0.02827124483883381,
      "5": 0.02508690394461155,
      "6": 0.022516116499900818,
      "7": 0.0706554725766182,
      "8": 0.07568445056676865,
      "9": 0.05779770761728287,
      "10": 0.04797745496034622,
      "11": 0.06330781430006027,
      "12": 0.042975474148988724,
      "13": 0.057478226721286774,
      "14": 0.06288282573223114,
      "15": 0.05868130549788475,
      "16": 0.060786861926317215,
      "17": 0.03845611587166786,
      "18": 0.03723089396953583,
      "19": 0.08863607794046402
    },
    "text": "the universe mirrored in a puddleisnt it amazing how there always seems to be another bottle of bheer therealeph one bottles of beer on the wall aleph one null bottles of beer\tyou too are a puddle\tas above so below",
    "id": 1
  },
 

In [55]:
import boto3
import json
client = boto3.client('sagemaker-runtime')

response = client.invoke_endpoint(
EndpointName=ENDPOINT_NAME,
ContentType="application/json",
Accept="application/json",
TargetModel="model1.tar.gz",
Body=json.dumps(v)
)
print(response['Body'].read().decode())

[
  {
    "score": 0.08863607794046402,
    "Categ": 19,
    "pred": "n1",
    "probabs": {
      "0": 0.0542377345263958,
      "1": 0.04734285920858383,
      "2": 0.02886742539703846,
      "3": 0.03112703375518322,
      "4": 0.02827124483883381,
      "5": 0.02508690394461155,
      "6": 0.022516116499900818,
      "7": 0.0706554725766182,
      "8": 0.07568445056676865,
      "9": 0.05779770761728287,
      "10": 0.04797745496034622,
      "11": 0.06330781430006027,
      "12": 0.042975474148988724,
      "13": 0.057478226721286774,
      "14": 0.06288282573223114,
      "15": 0.05868130549788475,
      "16": 0.060786861926317215,
      "17": 0.03845611587166786,
      "18": 0.03723089396953583,
      "19": 0.08863607794046402
    },
    "text": "the universe mirrored in a puddleisnt it amazing how there always seems to be another bottle of bheer therealeph one bottles of beer on the wall aleph one null bottles of beer\tyou too are a puddle\tas above so below",
    "id": 1
  },
 